<a href="https://colab.research.google.com/github/Syed8855/Titanic_Dataset_Profile_Repo/blob/main/Talk_to_Your_Data_Building_a_Natural_Language_to_SQL_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Objective:**
1. Generate new datasets for employees and an updated orders table using Mockaroo.

2. Create the new table in our SQLite database and load all the data.

3. Update our master prompt to include the new table schema.

4. Test the system with new questions related to employees.

### Key Takeaways
* Prompt Engineering Scalability: Detailed, well-structured prompts with schemas, roles, and examples are critical for success.
* Democratizing Data Access: Enables non-technical users to query databases with natural language.
* Flexibility: Schema updates don’t require retraining—only prompt updates.
* Handling Ambiguity: Can be instructed to ask clarifying questions, improving usability.

Importing Datasets From `Mockaroo`

In [1]:
! curl "https://api.mockaroo.com/api/dde01370?count=1000&key=11149690" > "customers.csv"
! curl "https://api.mockaroo.com/api/8ba6f630?count=1000&key=11149690" > "products.csv"



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 98655    0 98655    0     0  51213      0 --:--:--  0:00:01 --:--:-- 51196
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  682k    0  682k    0     0   287k      0 --:--:--  0:00:02 --:--:--  287k


In [2]:
! curl "https://api.mockaroo.com/api/9bb4af90?count=200&key=12883420" > "Employees_Data.csv"
! curl "https://api.mockaroo.com/api/8edb9c10?count=1000&key=12883420" > "Orders_Data.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19059    0 19059    0     0  26532      0 --:--:-- --:--:-- --:--:-- 26544
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70226    0 70226    0     0  42713      0 --:--:--  0:00:01 --:--:-- 42690


In [3]:
!ls -1 *.csv

customers.csv
Employees_Data.csv
Orders_Data.csv
products.csv


## DataBase Setup

In [4]:
import sqlite3
import pandas as pd
import os

customers_schema = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id INT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50),
    email VARCHAR(50), phone_number VARCHAR(50), address VARCHAR(50),
    city VARCHAR(50), country VARCHAR(50), postal_code VARCHAR(50), loyalty_points INT
);
"""

products_schema = """
CREATE TABLE IF NOT EXISTS products (
    product_id INT PRIMARY KEY, product_name TEXT, description TEXT, price DECIMAL(10,2),
    discount_percentage DECIMAL(5,2), category VARCHAR(50), brand TEXT, stock_quantity INT,
    color VARCHAR(50), size VARCHAR(20), weight DECIMAL(5,2), dimensions TEXT,
    release_date DATE, rating DECIMAL(3,1), reviews_count INT, seller_name TEXT,
    seller_rating DECIMAL(3,1), seller_reviews_count INT, shipping_method VARCHAR(20),
    shipping_cost DECIMAL(6,2)
);
"""

orders_schema = """
CREATE TABLE IF NOT EXISTS orders (
    order_id INT PRIMARY KEY, customer_id INT, product_id INT, quantity INT,
    unit_price DECIMAL(10,2), total_price DECIMAL(10,2), order_date DATE,
    shipping_address VARCHAR(255), payment_method VARCHAR(20), status VARCHAR(20),
    employee_id INT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
);
"""

employees_schema = """
CREATE TABLE IF NOT EXISTS employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    job_title VARCHAR(50),
    department VARCHAR(50),
    hire_date DATE,
    salary INT,
    manager_id INT
);
"""


COLUMN_DATA_TYPES = {
    'customers': { 'customer_id': 'int64', 'loyalty_points': 'int64' },
    'products': { 'product_id': 'int64', 'price': 'float64', 'discount_percentage': 'float64', 'stock_quantity': 'int64', 'weight': 'float64', 'release_date': 'datetime64[ns]', 'rating': 'float64', 'reviews_count': 'int64', 'seller_rating': 'float64', 'seller_reviews_count': 'int64', 'shipping_cost': 'float64' },
    'orders': { 'order_id': 'int64', 'customer_id': 'int64', 'product_id': 'int64', 'quantity': 'int64', 'unit_price': 'float64', 'total_price': 'float64', 'order_date': 'datetime64[ns]', 'employee_id': 'int64' },
    'employees': { 'employee_id': 'int64', 'hire_date': 'datetime64[ns]', 'salary': 'int64', 'manager_id': 'Int64' } # Use Int64 for nullable integers
}

db_name = 'ecommerce_with_employees.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

conn = None
try:
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created.")

     # Create all tables
    cursor.execute(customers_schema)
    cursor.execute(products_schema)
    cursor.execute(employees_schema) # Create employees before orders due to foreign key
    cursor.execute(orders_schema)
    print("All tables created successfully.")

    # Map CSVs to table names and schemas
    csv_to_table_map = {
        'customers.csv': ('customers', COLUMN_DATA_TYPES['customers']),
        'products.csv': ('products', COLUMN_DATA_TYPES['products']),
        'Orders_Data.csv': ('orders', COLUMN_DATA_TYPES['orders']),
        'Employees_Data.csv': ('employees', COLUMN_DATA_TYPES['employees'])
    }

    for csv_file, (table_name, dtypes) in csv_to_table_map.items():
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
            for col, dtype in dtypes.items():
                if 'datetime' in dtype:
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                else:
                    df[col] = pd.to_numeric(df[col], errors='coerce').astype(dtype)

            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{table_name}'.")
        else:
            print(f"Warning: '{csv_file}' not found.")

    conn.commit()
    print("\nData committed successfully.")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if conn:
        conn.close()
        print("Database connection closed.")


Database 'ecommerce_with_employees.db' created.
All tables created successfully.
  -> Data from 'customers.csv' loaded into 'customers'.
  -> Data from 'products.csv' loaded into 'products'.
  -> Data from 'Orders_Data.csv' loaded into 'orders'.
  -> Data from 'Employees_Data.csv' loaded into 'employees'.

Data committed successfully.
Database connection closed.


### Setting Up The AI Client

In [5]:
!pip install -q google-generativeai

from google import genai
from google.colab import userdata
import json

# Configure your API key
genai_client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

### Master prompt

In [6]:
updated_prompt = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for a business intelligence dashboard. The database dialect is always **SQLite**.

The database consists of the following four tables:

**customers table:**
```sql
CREATE TABLE customers (
    customer_id INT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50),
    email VARCHAR(50), phone_number VARCHAR(50), address VARCHAR(50),
    city VARCHAR(50), country VARCHAR(50), postal_code VARCHAR(50), loyalty_points INT
);
```

** products table:**
```sql
CREATE TABLE products (
    product_id INT PRIMARY KEY, product_name TEXT, description TEXT, price DECIMAL(10,2),
    discount_percentage DECIMAL(5,2), category VARCHAR(50), brand TEXT, stock_quantity INT,
    color VARCHAR(50), size VARCHAR(20), weight DECIMAL(5,2), dimensions TEXT,
    release_date DATE, rating DECIMAL(3,1), reviews_count INT, seller_name TEXT,
    seller_rating DECIMAL(3,1), seller_reviews_count INT, shipping_method VARCHAR(20),
    shipping_cost DECIMAL(6,2)
);
```
**orders table:**
```sql
CREATE TABLE orders (
    order_id INT PRIMARY KEY, customer_id INT, product_id INT, quantity INT,
    unit_price DECIMAL(10,2), total_price DECIMAL(10,2), order_date DATE,
    shipping_address VARCHAR(255), payment_method VARCHAR(20), status VARCHAR(20),
    employee_id INT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
);
```

**employees table:**
```sql
CREATE TABLE employees (
    employee_id INT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50),
    email VARCHAR(50), job_title VARCHAR(50), department VARCHAR(50),
    hire_date DATE, salary INT, manager_id INT
);
```

-----

### **TASK**

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1.  **Analyze the User's Query:** Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like `SUM`, `COUNT`, `AVG`), and ordering they are asking for.
2.  **Map to the Schema:** Map the entities from the user's query to the appropriate tables (`customers`, `products`, `orders`) and columns. Determine the necessary `JOIN` operations using `customers.customer_id` and `products.product_id` as foreign keys in the `orders` table.
3.  **Construct the SQLite Query:** Write a clean and efficient `SELECT` statement that is syntactically correct for SQLite. Ensure all table and column names are accurate.
4.  **Handle Ambiguity:** If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### **EXAMPLES**

**Example 1:**

  * **User Query:** "What are the names of the top 3 products with the highest total revenue?"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT T2.product_name FROM orders AS T1 INNER JOIN products AS T2 ON T1.product_id = T2.product_id GROUP BY T2.product_name ORDER BY SUM(T1.total_price) DESC LIMIT 3;"
    }
    ```

**Example 2: **

  * **User Query:** "Who is the highest paid employee?"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT first_name, last_name, salary FROM employees ORDER BY salary DESC LIMIT 1;"
    }
    ```

-----

### **OUTPUT FORMAT**

Your final response must be a single JSON object with two keys:

1.  `"status"`: A string with one of three possible values: `"success"`, `"clarification_needed"`, or `"error"`.
2.  `"response": A string containing the SQL query, clarifying question, or error message.

"""


Defining the Helper Functions

In [7]:
def get_sql_query(genai_client, prompt, user_query):
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-1.5-flash', contents=contents)

  clean_response = response.text.replace('```json', '').replace('```', '').strip()
  try:
    output = json.loads(clean_response)
  except json.JSONDecodeError:
    print(f"ERROR: The AI returned a non-JSON response: \n{clean_response}\n---")
    output = {"status": "error", "response": "The AI model did not return a valid JSON format."}
  return output

def execute_query(query, db_name='ecommerce_with_employees.db'):

    conn = None
    try:
        conn = sqlite3.connect(db_name)
        print(f"\nExecuting query on '{db_name}':\n{query}")
        results_df = pd.read_sql_query(query, conn)
        print("Query executed successfully.")
        return results_df
    except Exception as e:
        print(f"Error executing query: {e}")
        return None
    finally:
        if conn:
            conn.close()

def text2sql(genai_client, prompt, user_query):

  print(f"Translating your question: '{user_query}'")
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  else:
    print(f"\nReceived a non-success status: {output['status']}")
    print(f"Response: {output['response']}")
    return None

### Questionair

In [8]:
print("Running your Text-to-SQL query")
user_question = "Who are the top 5 highest-paid employees?"
result = text2sql(genai_client, updated_prompt, user_question)

result

Running your Text-to-SQL query
Translating your question: 'Who are the top 5 highest-paid employees?'

Executing query on 'ecommerce_with_employees.db':
SELECT first_name, last_name, salary FROM employees ORDER BY salary DESC LIMIT 5;
Query executed successfully.


,first_name,last_name,salary
0,Brodie,Miko,990102
1,Cris,Vidgen,985688
2,Balduin,Seys,980425
3,Maddy,Lescop,968204
4,Brittney,Ellwell,965026


In [9]:
print("Running your Text-to-SQL query")
user_question = "What are the names of the top 3 products with the highest total revenue?"
result = text2sql(genai_client, updated_prompt, user_question)

result

Running your Text-to-SQL query
Translating your question: 'What are the names of the top 3 products with the highest total revenue?'

Executing query on 'ecommerce_with_employees.db':
SELECT T2.product_name FROM orders AS T1 INNER JOIN products AS T2 ON T1.product_id = T2.product_id GROUP BY T2.product_name ORDER BY SUM(T1.total_price) DESC LIMIT 3;
Query executed successfully.


,product_name
0,porttitor lorem id ligula suspendisse ornare c...
1,ultrices erat tortor sollicitudin mi sit amet ...
2,dictumst morbi vestibulum velit id pretium iac...


In [11]:
print("Running your Text-to-SQL query")
user_question = "Show me the total number of orders handled by each department."
result = text2sql(genai_client, updated_prompt, user_question)

result

Running your Text-to-SQL query
Translating your question: 'Show me the total number of orders handled by each department.'

Executing query on 'ecommerce_with_employees.db':
SELECT e.department, COUNT(o.order_id) AS total_orders FROM employees e JOIN orders o ON e.employee_id = o.employee_id GROUP BY e.department;
Query executed successfully.


,department,total_orders
0,Accounting,68
1,Business Development,86
2,Engineering,91
3,Human Resources,114
4,Legal,78
5,Marketing,77
6,Product Management,59
7,Research and Development,96
8,Sales,61
9,Services,97


In [10]:
print("Running your Text-to-SQL query")
user_question = "Which employee is responsible for the highest value of sales? Show their name and the total sales amount."
result = text2sql(genai_client, updated_prompt, user_question)

result

Running your Text-to-SQL query
Translating your question: 'Which employee is responsible for the highest value of sales? Show their name and the total sales amount.'

Executing query on 'ecommerce_with_employees.db':
SELECT e.first_name, e.last_name, SUM(o.total_price) AS total_sales FROM employees e JOIN orders o ON e.employee_id = o.employee_id GROUP BY e.employee_id ORDER BY total_sales DESC LIMIT 1;
Query executed successfully.


,first_name,last_name,total_sales
0,Brucie,Boskell,11034.21


In [13]:
print("Running your Text-to-SQL query")
user_question = "Which department has the highest average salary?"
result = text2sql(genai_client, updated_prompt, user_question)

result

Running your Text-to-SQL query
Translating your question: 'Which department has the highest average salary?'

Executing query on 'ecommerce_with_employees.db':
SELECT department, AVG(salary) AS average_salary FROM employees GROUP BY department ORDER BY average_salary DESC LIMIT 1;
Query executed successfully.


,department,average_salary
0,Legal,624271.733333


## **Conclusion**
1. Successfully demonstrated the creation of a Text-to-SQL generator using prompt engineering.
2. By including the database schema, instructions, and examples in the prompt, the AI accurately translated natural language into SQLite queries.
3. Extended functionality by adding a new employees table and updating the orders table, proving scalability and adaptability.
4. The model handled both simple single-table queries and complex JOIN operations across multiple tables.


---

